In [4]:
!pip install fireworks-ai pandas scikit-learn

  Using cached pandas-2.0.3-cp38-cp38-macosx_11_0_arm64.whl (10.7 MB)
  Using cached scikit_learn-1.3.2-cp38-cp38-macosx_12_0_arm64.whl (9.4 MB)
You should consider upgrading via the '/Users/scottkramer/.pyenv/versions/3.8.16/envs/fine-tuning-workshop/bin/python3.8 -m pip install --upgrade pip' command.


MAKE SURE YOU FOLLOW THE INSTRUCTIONS TO INSTALL FIRECTL, BASED ON YOUR ARCHITECTURE
https://readme.fireworks.ai/reference/installation-1

In [2]:
# BEFORE PROCEEDING, MAKE SURE YOU FOLLOW THE INSTRUCTIONS TO INSTALL FIRECTL, BASED ON YOUR ARCHITECTURE 
# https://readme.fireworks.ai/reference/installation-1

# sign in to firectl
!firectl signin

Signed in as: sdkramer10@gmail.com
Account ID: sdkramer10-5e98cb


In [5]:
import json
import os

import fireworks.client as fc
from fireworks.client import Fireworks
import pandas as pd
from sklearn.model_selection import train_test_split

In [6]:
# Set this to your Fireworks account id
account_id = 'sdkramer10-5e98cb'

# Uncomment the line below and set the value to your account's API key
# fc.api_key = '<API KEY>'

client = Fireworks()

In [7]:
# Load the csv dataset from google drive
item_class_csv_url = 'item_classification.csv'
df = pd.read_csv(item_class_csv_url)

# Perform a train/test split of the data
x_train, x_test, y_train, y_test = train_test_split( df['Scope'], df['Class'], test_size=0.2, random_state=42)

In [63]:
possible_classes = '\n'.join(list(y_train.unique()))
system_message = f'''Classify the following scope. Respond with ONLY the name of the appropriate class from the list below.

CLASSES:
{possible_classes}'''

print(system_message)

Classify the following scope. Respond with ONLY the name of the appropriate class from the list below.

CLASSES:
Pressure Safety Device
Piping
Structure
Pressure Vessel (VIE)
FU Items
Non Structural Tank
Campaign
Lifting 
Corrosion Monitoring
Pressure Vessel (VII)
Lifting
Flare TIP
Flame Arrestor
Flare Tip
Intelligent Pigging


In [79]:
json_objs = list()
for i in range(len(x_train)):
    msg = {"messages": [
        {"role": "system", "content": system_message}, 
        {"role": "user", "content": x_train.iloc[i]}, 
        {"role": "assistant", "content": y_train.iloc[i]}
    ]}  

    json_objs.append(msg)

dataset_file_name = 'item_classification.jsonl'
dataset_name = 'item-class-v4'

with open(dataset_file_name, 'w') as f:
    for obj in json_objs:
        json.dump(obj, f)
        f.write('\n')

In [80]:
!# Upload our dataset to fireworks
!firectl create dataset {dataset_name} {dataset_file_name}

643.77 KiB / 643.77 KiB [---------------------------] 100.00% 3.98 MiB p/s 400ms


In [81]:
!firectl create fine-tuning-job --settings-file item_classification.yaml  --display-name {dataset_name} --dataset {dataset_name} 

Name: accounts/sdkramer10-5e98cb/fineTuningJobs/c55a772445ed48f1bbad2b15c1508b86
Display Name: item-class-v4
Create Time: 2024-07-09 17:52:30
State: CREATING
Dataset: accounts/sdkramer10-5e98cb/datasets/item-class-v4
Created By: sdkramer10@gmail.com
Container Version: 
Model Id: 
Wandb Url: 
Conversation:
  Jinja Template: {%- set _mode = mode | default('generate', true) -%}
{%- set stop_token = '<|eot_id|>' -%}
{%- set message_roles = ['SYSTEM', 'USER', 'ASSISTANT'] -%}
{%- set ns = namespace(initial_system_message_handled=false, last_assistant_index_for_eos=-1, messages=messages) -%}
{%- for message in ns.messages -%}
    {%- if not message.get('role') -%}
        {{ raise_exception('Key [role] is missing. Original input: ' +  message|tojson) }}
    {%- endif -%}
    {%- if message['role'] | upper not in message_roles -%}
        {{ raise_exception('Invalid role ' + message['role']|tojson + '. Only ' + message_roles|tojson + ' are supported.') }}
    {%- endif -%}
    {%- if 'content

In [83]:
# replace model_id with the id of your fine-tuned model.
# In the output from the cell above, the first line is "Name: accounts/<ACCOUNT_ID>/fineTuningJobs/<MODEL_ID>"
model_id = 'c55a772445ed48f1bbad2b15c1508b86'

In [84]:
# Get the status of the fine-tuning job. Wait until the state says COMPLETED before continuing (~10-20 mins).
!firectl get fine-tuning-job {model_id}

Name: accounts/sdkramer10-5e98cb/fineTuningJobs/c55a772445ed48f1bbad2b15c1508b86
Display Name: item-class-v4
Create Time: 2024-07-09 17:52:30
State: COMPLETED
Dataset: accounts/sdkramer10-5e98cb/datasets/item-class-v4
Status: OK
Created By: sdkramer10@gmail.com
Container Version: 
Model Id: 
Wandb Url: 
Conversation:
  Jinja Template: {%- set _mode = mode | default('generate', true) -%}
{%- set stop_token = '<|eot_id|>' -%}
{%- set message_roles = ['SYSTEM', 'USER', 'ASSISTANT'] -%}
{%- set ns = namespace(initial_system_message_handled=false, last_assistant_index_for_eos=-1, messages=messages) -%}
{%- for message in ns.messages -%}
    {%- if not message.get('role') -%}
        {{ raise_exception('Key [role] is missing. Original input: ' +  message|tojson) }}
    {%- endif -%}
    {%- if message['role'] | upper not in message_roles -%}
        {{ raise_exception('Invalid role ' + message['role']|tojson + '. Only ' + message_roles|tojson + ' are supported.') }}
    {%- endif -%}
    {%-

In [85]:
!firectl deploy {model_id}

In [87]:
# Get the model's status. Wait until the "Deployed Model Refs" says DEPLOYED before running reference (~15-20 mins).
!firectl get model {model_id}

Name: accounts/sdkramer10-5e98cb/models/c55a772445ed48f1bbad2b15c1508b86
Display Name: 
Description: 
Create Time: 2024-07-09 18:06:43
Created By: 
State: READY
Status: OK
Kind: HF_PEFT_ADDON
Deployment Id: 
Github Url: 
Hugging Face Url: 
Peft Details:
  Base Model: accounts/fireworks/models/llama-v3-8b-instruct-hf
  R: 32
  Target Modules: [
    down_proj, 
    up_proj, 
    q_proj, 
    o_proj, 
    v_proj, 
    k_proj, 
    gate_proj
  ]
Public: false
Conversation Config:
  Style: jinja
  System: 
  Template: 
Context Length: 8192
Supports Image Input: false
Supports Tools: false
Imported From: 
Fine Tuning Job: accounts/sdkramer10-5e98cb/fineTuningJobs/c55a772445ed48f1bbad2b15c1508b86
Default Draft Model: 
Default Draft Token Count: 0
Precisions: []
Deployed Model Refs: [
  {
    Name: accounts/sdkramer10-5e98cb/deployedModels/c55a772445ed48f1bbad2b15c1508b86-6ffe0b4b
    Deployment: accounts/fireworks/deployments/052a4161
    State: DEPLOYED
    Default: true
  }
]


In [88]:
ft_model_name = f'accounts/{account_id}/models/{model_id}'

In [91]:
def generate_results(scopes):
    responses = list()
    
    for scope in scopes:
            
        msg = [
              {"role": "system", "content": system_message},
              {"role": "user", "content": scope}
        ]
        response = client.chat.completions.create(
            model=ft_model_name,
            messages=msg,
            temperature=0,
        )
    
        response = response.choices[0].message.content
        responses.append(response) 

    return responses

In [92]:
predictions = generate_results(x_test.tolist())

In [97]:
num_correct = len([i for i in range(len(predictions)) if predictions[i] == y_test.iloc[i]])
total = len(predictions)
pct_accuracy = round(100 * num_correct / total, 2)
print(f'Test Set Accuracy: {pct_accuracy}')

Test Set Accuracy: 98.23


In [ ]:
# OPTIONAL: undeploy the model. Fireworks currently does not charge for deploy models, only for usage, so up to you
# whether you want to keep it deployed
!fireworks undeploy {model_id}